In [ ]:
# Core imports for the entire project
import numpy as np
import random
import copy
import math
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, deque
from typing import List, Dict, Tuple, Optional, Any
from dataclasses import dataclass, field
from enum import Enum
import json
import time

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)

print("Environment setup complete!")
print(f"NumPy version: {np.__version__}")
print(f"Python random module loaded")
print(f"Ready to build Warhammer AI system")


In [ ]:
class UnitType(Enum):
    CHARACTER = "Character"
    CORE = "Core"
    SPECIAL = "Special"
    RARE = "Rare"

class WeaponType(Enum):
    MELEE = "Melee"
    RANGED = "Ranged"
    ARTILLERY = "Artillery"

@dataclass
class Equipment:
    name: str
    weapon_type: WeaponType
    strength_modifier: int = 0
    to_hit_modifier: int = 0
    range_inches: int = 0  # 0 for melee
    special_rules: List[str] = field(default_factory=list)
    points_cost: int = 0

@dataclass
class Unit:
    name: str
    unit_type: UnitType
    # Warhammer characteristics
    movement: int  # M
    weapon_skill: int  # WS
    ballistic_skill: int  # BS
    strength: int  # S
    toughness: int  # T
    wounds: int  # W
    initiative: int  # I
    attacks: int  # A
    leadership: int  # Ld
    armour_save: int  # AS (target number needed on D6)
    
    # Game state
    current_wounds: int = None
    position: Tuple[int, int] = (0, 0)
    equipment: List[Equipment] = field(default_factory=list)
    special_rules: List[str] = field(default_factory=list)
    points_cost: int = 0
    models_count: int = 1
    current_models: int = None
    
    def __post_init__(self):
        if self.current_wounds is None:
            self.current_wounds = self.wounds
        if self.current_models is None:
            self.current_models = self.models_count
    
    def is_alive(self) -> bool:
        return self.current_models > 0 and self.current_wounds > 0
    
    def take_wounds(self, wounds: int) -> int:
        """Apply wounds and return models removed"""
        models_removed = 0
        remaining_wounds = wounds
        
        while remaining_wounds > 0 and self.current_models > 0:
            wounds_to_apply = min(remaining_wounds, self.current_wounds)
            self.current_wounds -= wounds_to_apply
            remaining_wounds -= wounds_to_apply
            
            if self.current_wounds <= 0:
                models_removed += 1
                self.current_models -= 1
                self.current_wounds = self.wounds  # Next model starts with full wounds
        
        return models_removed

    def get_effective_characteristic(self, characteristic: str) -> int:
        """Get characteristic with equipment modifiers"""
        base_value = getattr(self, characteristic)
        
        # Apply equipment modifiers
        for equipment in self.equipment:
            if characteristic == "strength" and hasattr(equipment, 'strength_modifier'):
                base_value += equipment.strength_modifier
        
        return max(1, base_value)  # Minimum of 1


In [ ]:
@dataclass
class Board:
    width: int = 48  # inches, scaled down
    height: int = 72  # inches, scaled down
    terrain: np.ndarray = field(default_factory=lambda: np.zeros((48, 72)))
    
    def __post_init__(self):
        if self.terrain.size == 0:
            self.terrain = np.zeros((self.width, self.height))
    
    def is_valid_position(self, x: int, y: int) -> bool:
        return 0 <= x < self.width and 0 <= y < self.height
    
    def distance(self, pos1: Tuple[int, int], pos2: Tuple[int, int]) -> float:
        return math.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)
    
    def has_line_of_sight(self, pos1: Tuple[int, int], pos2: Tuple[int, int]) -> bool:
        """Simplified line of sight - just check for direct line"""
        # For now, assume clear line of sight if within range
        # Could be enhanced with terrain blocking
        return True

class GamePhase(Enum):
    STRATEGY = "Strategy"
    MOVEMENT = "Movement"
    SHOOTING = "Shooting"
    COMBAT = "Combat"
    END = "End"

@dataclass
class GameState:
    board: Board
    player1_army: List[Unit]
    player2_army: List[Unit]
    current_player: int = 1  # 1 or 2
    current_phase: GamePhase = GamePhase.STRATEGY
    round_number: int = 1
    max_rounds: int = 6
    game_over: bool = False
    winner: Optional[int] = None
    
    def get_current_army(self) -> List[Unit]:
        return self.player1_army if self.current_player == 1 else self.player2_army
    
    def get_enemy_army(self) -> List[Unit]:
        return self.player2_army if self.current_player == 1 else self.player1_army
    
    def get_alive_units(self, army: List[Unit]) -> List[Unit]:
        return [unit for unit in army if unit.is_alive()]
    
    def is_game_over(self) -> bool:
        if self.game_over:
            return True
        
        # Check if any army is destroyed
        p1_alive = len(self.get_alive_units(self.player1_army)) > 0
        p2_alive = len(self.get_alive_units(self.player2_army)) > 0
        
        if not p1_alive and not p2_alive:
            self.game_over = True
            self.winner = None  # Draw
        elif not p1_alive:
            self.game_over = True
            self.winner = 2
        elif not p2_alive:
            self.game_over = True
            self.winner = 1
        elif self.round_number > self.max_rounds:
            self.game_over = True
            # Determine winner by points remaining
            p1_points = sum(unit.points_cost for unit in self.get_alive_units(self.player1_army))
            p2_points = sum(unit.points_cost for unit in self.get_alive_units(self.player2_army))
            
            if p1_points > p2_points:
                self.winner = 1
            elif p2_points > p1_points:
                self.winner = 2
            else:
                self.winner = None  # Draw
        
        return self.game_over
    
    def advance_phase(self):
        """Advance to next phase/turn"""
        if self.current_phase == GamePhase.STRATEGY:
            self.current_phase = GamePhase.MOVEMENT
        elif self.current_phase == GamePhase.MOVEMENT:
            self.current_phase = GamePhase.SHOOTING
        elif self.current_phase == GamePhase.SHOOTING:
            self.current_phase = GamePhase.COMBAT
        elif self.current_phase == GamePhase.COMBAT:
            self.current_phase = GamePhase.END
        else:  # END phase
            # Switch to other player
            if self.current_player == 1:
                self.current_player = 2
            else:
                self.current_player = 1
                self.round_number += 1
            self.current_phase = GamePhase.STRATEGY

print("Core game classes defined successfully!")


In [ ]:
# Define Nuln Units and Equipment

def create_nuln_equipment():
    """Create Nuln-specific equipment"""
    equipment = {
        "handgun": Equipment(
            name="Handgun",
            weapon_type=WeaponType.RANGED,
            strength_modifier=1,
            range_inches=24,
            special_rules=["Armor_Piercing"],
            points_cost=2
        ),
        "halberd": Equipment(
            name="Halberd",
            weapon_type=WeaponType.MELEE,
            strength_modifier=1,
            special_rules=["Strikes_First"],
            points_cost=1
        ),
        "great_cannon": Equipment(
            name="Great Cannon",
            weapon_type=WeaponType.ARTILLERY,
            strength_modifier=6,
            range_inches=48,
            special_rules=["Artillery", "Guess_Range"],
            points_cost=0  # Included in unit cost
        ),
        "engineer_tools": Equipment(
            name="Engineer Tools",
            weapon_type=WeaponType.MELEE,
            special_rules=["Engineer_Buff_Artillery"],
            points_cost=5
        )
    }
    return equipment

def create_nuln_units():
    """Create the core Nuln army units"""
    equipment = create_nuln_equipment()
    
    units = {
        "nuln_state_troops_handgun": Unit(
            name="Nuln State Troops (Handguns)",
            unit_type=UnitType.CORE,
            movement=4, weapon_skill=3, ballistic_skill=3,
            strength=3, toughness=3, wounds=1,
            initiative=3, attacks=1, leadership=7,
            armour_save=5,  # Light armor
            equipment=[equipment["handgun"]],
            points_cost=12,
            models_count=10
        ),
        
        "nuln_state_troops_halberd": Unit(
            name="Nuln State Troops (Halberds)",
            unit_type=UnitType.CORE,
            movement=4, weapon_skill=3, ballistic_skill=3,
            strength=3, toughness=3, wounds=1,
            initiative=3, attacks=1, leadership=7,
            armour_save=5,  # Light armor
            equipment=[equipment["halberd"]],
            points_cost=10,
            models_count=10
        ),
        
        "nuln_great_cannon": Unit(
            name="Nuln Great Cannon",
            unit_type=UnitType.RARE,
            movement=0, weapon_skill=0, ballistic_skill=3,
            strength=7, toughness=7, wounds=3,
            initiative=1, attacks=0, leadership=0,
            armour_save=7,  # No save
            equipment=[equipment["great_cannon"]],
            special_rules=["War_Machine"],
            points_cost=120,
            models_count=1
        ),
        
        "nuln_engineer": Unit(
            name="Nuln Engineer",
            unit_type=UnitType.CHARACTER,
            movement=4, weapon_skill=3, ballistic_skill=4,
            strength=3, toughness=3, wounds=2,
            initiative=3, attacks=1, leadership=8,
            armour_save=5,  # Light armor
            equipment=[equipment["engineer_tools"]],
            special_rules=["Engineer"],
            points_cost=65,
            models_count=1
        )
    }
    
    return units

def create_enemy_units():
    """Create simple enemy units for testing"""
    units = {
        "orc_warriors": Unit(
            name="Orc Warriors",
            unit_type=UnitType.CORE,
            movement=4, weapon_skill=3, ballistic_skill=3,
            strength=3, toughness=4, wounds=1,
            initiative=2, attacks=1, leadership=7,
            armour_save=6,  # Light armor + shield
            points_cost=6,
            models_count=15
        ),
        
        "orc_warboss": Unit(
            name="Orc Warboss",
            unit_type=UnitType.CHARACTER,
            movement=4, weapon_skill=5, ballistic_skill=3,
            strength=4, toughness=4, wounds=3,
            initiative=3, attacks=3, leadership=8,
            armour_save=4,  # Heavy armor
            special_rules=["Leader"],
            points_cost=100,
            models_count=1
        )
    }
    
    return units

nuln_units = create_nuln_units()
enemy_units = create_enemy_units()

print("Nuln and enemy units created!")
print(f"Nuln units available: {list(nuln_units.keys())}")
print(f"Enemy units available: {list(enemy_units.keys())}")


In [ ]:
class GameMechanics:
    """Core game mechanics for Warhammer combat"""
    
    @staticmethod
    def roll_d6() -> int:
        return random.randint(1, 6)
    
    @staticmethod
    def roll_multiple_d6(count: int) -> List[int]:
        return [GameMechanics.roll_d6() for _ in range(count)]
    
    @staticmethod
    def leadership_test(unit: Unit, modifier: int = 0) -> bool:
        """Roll 2D6 vs Leadership"""
        roll = GameMechanics.roll_d6() + GameMechanics.roll_d6()
        return roll <= (unit.leadership + modifier)
    
    @staticmethod
    def shooting_attack(shooter: Unit, target: Unit, distance: float) -> int:
        """Resolve shooting attack, return wounds caused"""
        if not shooter.is_alive() or not target.is_alive():
            return 0
        
        # Get ranged weapon
        ranged_weapons = [eq for eq in shooter.equipment if eq.weapon_type == WeaponType.RANGED or eq.weapon_type == WeaponType.ARTILLERY]
        if not ranged_weapons:
            return 0
        
        weapon = ranged_weapons[0]
        
        # Check range
        if distance > weapon.range_inches:
            return 0
        
        # Calculate number of shots
        if weapon.weapon_type == WeaponType.ARTILLERY:
            shots = 1  # Cannons fire once
        else:
            shots = shooter.current_models  # Each model shoots once
        
        wounds_caused = 0
        
        for _ in range(shots):
            # To Hit roll
            to_hit_roll = GameMechanics.roll_d6()
            to_hit_target = 7 - shooter.ballistic_skill
            
            # Apply modifiers
            if distance > weapon.range_inches // 2:  # Long range
                to_hit_target += 1
            
            if to_hit_roll < to_hit_target:
                continue  # Miss
            
            # To Wound roll
            wound_roll = GameMechanics.roll_d6()
            attacker_strength = shooter.strength + weapon.strength_modifier
            
            # Strength vs Toughness table (simplified)
            if attacker_strength >= target.toughness * 2:
                wound_target = 2
            elif attacker_strength > target.toughness:
                wound_target = 3
            elif attacker_strength == target.toughness:
                wound_target = 4
            elif attacker_strength < target.toughness:
                wound_target = 5
            else:  # Strength < Toughness/2
                wound_target = 6
            
            if wound_roll < wound_target:
                continue  # No wound
            
            # Armor save
            save_roll = GameMechanics.roll_d6()
            save_target = target.armour_save
            
            # Apply armor piercing
            if "Armor_Piercing" in weapon.special_rules:
                save_target += 1
            
            if save_roll < save_target:
                wounds_caused += 1
        
        return wounds_caused
    
    @staticmethod
    def melee_attack(attacker: Unit, defender: Unit) -> int:
        """Resolve melee attack, return wounds caused"""
        if not attacker.is_alive() or not defender.is_alive():
            return 0
        
        wounds_caused = 0
        attacks = attacker.attacks * attacker.current_models
        
        # Get melee weapon strength bonus
        strength_bonus = 0
        for weapon in attacker.equipment:
            if weapon.weapon_type == WeaponType.MELEE:
                strength_bonus = weapon.strength_modifier
                break
        
        for _ in range(attacks):
            # To Hit roll
            to_hit_roll = GameMechanics.roll_d6()
            
            # WS comparison (simplified)
            if attacker.weapon_skill >= defender.weapon_skill:
                to_hit_target = 4
            else:
                to_hit_target = 5
            
            if to_hit_roll < to_hit_target:
                continue  # Miss
            
            # To Wound roll (same as shooting)
            wound_roll = GameMechanics.roll_d6()
            attacker_strength = attacker.strength + strength_bonus
            
            if attacker_strength >= defender.toughness * 2:
                wound_target = 2
            elif attacker_strength > defender.toughness:
                wound_target = 3
            elif attacker_strength == defender.toughness:
                wound_target = 4
            elif attacker_strength < defender.toughness:
                wound_target = 5
            else:
                wound_target = 6
            
            if wound_roll < wound_target:
                continue  # No wound
            
            # Armor save
            save_roll = GameMechanics.roll_d6()
            if save_roll < defender.armour_save:
                wounds_caused += 1
        
        return wounds_caused

print("Game mechanics implemented!")


In [ ]:
from abc import ABC, abstractmethod

class GameAction(ABC):
    """Base class for all game actions"""
    
    @abstractmethod
    def is_valid(self, game_state: GameState) -> bool:
        pass
    
    @abstractmethod
    def execute(self, game_state: GameState) -> GameState:
        pass
    
    @abstractmethod
    def description(self) -> str:
        pass

class MoveAction(GameAction):
    def __init__(self, unit_index: int, new_position: Tuple[int, int]):
        self.unit_index = unit_index
        self.new_position = new_position
    
    def is_valid(self, game_state: GameState) -> bool:
        army = game_state.get_current_army()
        if self.unit_index >= len(army):
            return False
        
        unit = army[self.unit_index]
        if not unit.is_alive():
            return False
        
        # Check movement distance
        distance = game_state.board.distance(unit.position, self.new_position)
        if distance > unit.movement:
            return False
        
        # Check if position is valid
        return game_state.board.is_valid_position(self.new_position[0], self.new_position[1])
    
    def execute(self, game_state: GameState) -> GameState:
        new_state = copy.deepcopy(game_state)
        army = new_state.get_current_army()
        army[self.unit_index].position = self.new_position
        return new_state
    
    def description(self) -> str:
        return f"Move unit {self.unit_index} to {self.new_position}"

class ShootAction(GameAction):
    def __init__(self, shooter_index: int, target_index: int):
        self.shooter_index = shooter_index
        self.target_index = target_index
    
    def is_valid(self, game_state: GameState) -> bool:
        shooter_army = game_state.get_current_army()
        target_army = game_state.get_enemy_army()
        
        if (self.shooter_index >= len(shooter_army) or 
            self.target_index >= len(target_army)):
            return False
        
        shooter = shooter_army[self.shooter_index]
        target = target_army[self.target_index]
        
        if not shooter.is_alive() or not target.is_alive():
            return False
        
        # Check if unit has ranged weapons
        has_ranged = any(eq.weapon_type in [WeaponType.RANGED, WeaponType.ARTILLERY] 
                        for eq in shooter.equipment)
        
        return has_ranged
    
    def execute(self, game_state: GameState) -> GameState:
        new_state = copy.deepcopy(game_state)
        shooter_army = new_state.get_current_army()
        target_army = new_state.get_enemy_army()
        
        shooter = shooter_army[self.shooter_index]
        target = target_army[self.target_index]
        
        distance = new_state.board.distance(shooter.position, target.position)
        wounds = GameMechanics.shooting_attack(shooter, target, distance)
        
        if wounds > 0:
            target.take_wounds(wounds)
        
        return new_state
    
    def description(self) -> str:
        return f"Unit {self.shooter_index} shoots at unit {self.target_index}"

class ChargeAction(GameAction):
    def __init__(self, charger_index: int, target_index: int):
        self.charger_index = charger_index
        self.target_index = target_index
    
    def is_valid(self, game_state: GameState) -> bool:
        charger_army = game_state.get_current_army()
        target_army = game_state.get_enemy_army()
        
        if (self.charger_index >= len(charger_army) or 
            self.target_index >= len(target_army)):
            return False
        
        charger = charger_army[self.charger_index]
        target = target_army[self.target_index]
        
        if not charger.is_alive() or not target.is_alive():
            return False
        
        # Check charge distance (movement + D6)
        distance = game_state.board.distance(charger.position, target.position)
        max_charge_distance = charger.movement + 6  # Assume max roll
        
        return distance <= max_charge_distance
    
    def execute(self, game_state: GameState) -> GameState:
        new_state = copy.deepcopy(game_state)
        charger_army = new_state.get_current_army()
        target_army = new_state.get_enemy_army()
        
        charger = charger_army[self.charger_index]
        target = target_army[self.target_index]
        
        # Move charger adjacent to target
        target_x, target_y = target.position
        charger.position = (target_x + 1, target_y)  # Simplified positioning
        
        # Resolve combat
        attacker_wounds = GameMechanics.melee_attack(charger, target)
        defender_wounds = GameMechanics.melee_attack(target, charger)
        
        if attacker_wounds > 0:
            target.take_wounds(attacker_wounds)
        if defender_wounds > 0:
            charger.take_wounds(defender_wounds)
        
        return new_state
    
    def description(self) -> str:
        return f"Unit {self.charger_index} charges unit {self.target_index}"

class EndPhaseAction(GameAction):
    def is_valid(self, game_state: GameState) -> bool:
        return True
    
    def execute(self, game_state: GameState) -> GameState:
        new_state = copy.deepcopy(game_state)
        new_state.advance_phase()
        return new_state
    
    def description(self) -> str:
        return "End current phase"

print("Game actions implemented!")


In [ ]:
class MCTSNode:
    """Node in the MCTS tree"""
    
    def __init__(self, game_state: GameState, parent: Optional['MCTSNode'] = None, 
                 action: Optional[GameAction] = None):
        self.game_state = game_state
        self.parent = parent
        self.action = action  # Action that led to this state
        
        self.children: List['MCTSNode'] = []
        self.visits = 0
        self.wins = 0.0  # Total reward
        self.untried_actions: List[GameAction] = []
        self.fully_expanded = False
        
        # Initialize available actions
        self._initialize_actions()
    
    def _initialize_actions(self):
        """Initialize list of possible actions from this state"""
        self.untried_actions = self._get_possible_actions()
    
    def _get_possible_actions(self) -> List[GameAction]:
        """Get all valid actions from current state"""
        actions = []
        
        current_army = self.game_state.get_current_army()
        enemy_army = self.game_state.get_enemy_army()
        
        # Phase-specific actions
        if self.game_state.current_phase == GamePhase.MOVEMENT:
            # Movement actions
            for i, unit in enumerate(current_army):
                if unit.is_alive():
                    # Generate some movement options
                    for dx in [-unit.movement, 0, unit.movement]:
                        for dy in [-unit.movement, 0, unit.movement]:
                            new_x = unit.position[0] + dx
                            new_y = unit.position[1] + dy
                            action = MoveAction(i, (new_x, new_y))
                            if action.is_valid(self.game_state):
                                actions.append(action)
        
        elif self.game_state.current_phase == GamePhase.SHOOTING:
            # Shooting actions
            for i, shooter in enumerate(current_army):
                if shooter.is_alive():
                    for j, target in enumerate(enemy_army):
                        if target.is_alive():
                            action = ShootAction(i, j)
                            if action.is_valid(self.game_state):
                                actions.append(action)
        
        elif self.game_state.current_phase == GamePhase.COMBAT:
            # Charge actions
            for i, charger in enumerate(current_army):
                if charger.is_alive():
                    for j, target in enumerate(enemy_army):
                        if target.is_alive():
                            action = ChargeAction(i, j)
                            if action.is_valid(self.game_state):
                                actions.append(action)
        
        # Always can end phase
        actions.append(EndPhaseAction())
        
        return actions
    
    def is_fully_expanded(self) -> bool:
        """Check if all actions have been tried"""
        return len(self.untried_actions) == 0
    
    def is_terminal(self) -> bool:
        """Check if this is a terminal game state"""
        return self.game_state.is_game_over()
    
    def ucb1_value(self, exploration_constant: float = math.sqrt(2)) -> float:
        """Calculate UCB1 value for node selection"""
        if self.visits == 0:
            return float('inf')
        
        exploitation = self.wins / self.visits
        exploration = exploration_constant * math.sqrt(math.log(self.parent.visits) / self.visits)
        
        return exploitation + exploration
    
    def select_child(self) -> 'MCTSNode':
        """Select child with highest UCB1 value"""
        return max(self.children, key=lambda c: c.ucb1_value())
    
    def expand(self) -> 'MCTSNode':
        """Expand tree by adding a new child node"""
        if not self.untried_actions:
            return self
        
        action = self.untried_actions.pop()
        new_state = action.execute(self.game_state)
        child = MCTSNode(new_state, parent=self, action=action)
        self.children.append(child)
        
        return child
    
    def simulate(self) -> float:
        """Run random simulation from this node to terminal state"""
        current_state = copy.deepcopy(self.game_state)
        original_player = current_state.current_player
        
        # Random rollout
        while not current_state.is_game_over():
            actions = MCTSNode._get_actions_for_state(current_state)
            if not actions:
                break
            
            action = random.choice(actions)
            current_state = action.execute(current_state)
        
        # Return reward from perspective of original player
        if current_state.winner == original_player:
            return 1.0
        elif current_state.winner is None:
            return 0.5  # Draw
        else:
            return 0.0  # Loss
    
    @staticmethod
    def _get_actions_for_state(state: GameState) -> List[GameAction]:
        """Helper to get actions for a state (used in simulation)"""
        # Simplified action generation for simulation
        actions = []
        current_army = state.get_current_army()
        enemy_army = state.get_enemy_army()
        
        # Try a few basic actions per phase
        if state.current_phase == GamePhase.SHOOTING:
            for i, shooter in enumerate(current_army):
                if shooter.is_alive():
                    for j, target in enumerate(enemy_army):
                        if target.is_alive():
                            action = ShootAction(i, j)
                            if action.is_valid(state):
                                actions.append(action)
        
        elif state.current_phase == GamePhase.COMBAT:
            for i, charger in enumerate(current_army):
                if charger.is_alive():
                    for j, target in enumerate(enemy_army):
                        if target.is_alive():
                            action = ChargeAction(i, j)
                            if action.is_valid(state):
                                actions.append(action)
        
        # Always can end phase
        actions.append(EndPhaseAction())
        return actions
    
    def backpropagate(self, reward: float):
        """Backpropagate reward up the tree"""
        self.visits += 1
        self.wins += reward
        
        if self.parent:
            # Flip reward for opponent
            self.parent.backpropagate(1.0 - reward)

print("MCTS Node implementation complete!")


In [ ]:
class MCTSAgent:
    """Monte Carlo Tree Search agent for Warhammer"""
    
    def __init__(self, iterations: int = 1000, exploration_constant: float = math.sqrt(2)):
        self.iterations = iterations
        self.exploration_constant = exploration_constant
    
    def get_best_action(self, game_state: GameState) -> GameAction:
        """Get the best action using MCTS"""
        root = MCTSNode(game_state)
        
        # MCTS iterations
        for _ in range(self.iterations):
            # Selection
            node = self._select(root)
            
            # Expansion
            if not node.is_terminal() and not node.is_fully_expanded():
                node = node.expand()
            
            # Simulation
            reward = node.simulate()
            
            # Backpropagation
            node.backpropagate(reward)
        
        # Choose action with most visits
        if not root.children:
            # No children, return random action
            actions = root._get_possible_actions()
            return random.choice(actions) if actions else EndPhaseAction()
        
        best_child = max(root.children, key=lambda c: c.visits)
        return best_child.action
    
    def _select(self, node: MCTSNode) -> MCTSNode:
        """Selection phase - traverse tree using UCB1"""
        while not node.is_terminal():
            if not node.is_fully_expanded():
                return node
            else:
                node = node.select_child()
        return node

# Simple heuristic-based agent for comparison
class HeuristicAgent:
    """Simple heuristic-based agent"""
    
    def get_best_action(self, game_state: GameState) -> GameAction:
        """Get action using simple heuristics"""
        current_army = game_state.get_current_army()
        enemy_army = game_state.get_enemy_army()
        
        valid_actions = []
        
        # Phase-specific behavior
        if game_state.current_phase == GamePhase.SHOOTING:
            # Prioritize shooting with units that have ranged weapons
            for i, shooter in enumerate(current_army):
                if shooter.is_alive():
                    has_ranged = any(eq.weapon_type in [WeaponType.RANGED, WeaponType.ARTILLERY] 
                                   for eq in shooter.equipment)
                    if has_ranged:
                        # Find closest enemy
                        closest_target = None
                        closest_distance = float('inf')
                        for j, target in enumerate(enemy_army):
                            if target.is_alive():
                                distance = game_state.board.distance(shooter.position, target.position)
                                if distance < closest_distance:
                                    closest_distance = distance
                                    closest_target = j
                        
                        if closest_target is not None:
                            action = ShootAction(i, closest_target)
                            if action.is_valid(game_state):
                                valid_actions.append(action)
        
        elif game_state.current_phase == GamePhase.COMBAT:
            # Try to charge with melee units
            for i, charger in enumerate(current_army):
                if charger.is_alive():
                    # Find closest enemy to charge
                    for j, target in enumerate(enemy_army):
                        if target.is_alive():
                            action = ChargeAction(i, j)
                            if action.is_valid(game_state):
                                valid_actions.append(action)
                                break  # Only one charge per unit
        
        # If no specific actions, end phase
        if not valid_actions:
            return EndPhaseAction()
        
        return random.choice(valid_actions)

print("MCTS Agent implemented!")


In [ ]:
@dataclass
class ArmyComposition:
    """Represents a Nuln army composition (chromosome)"""
    units: List[str] = field(default_factory=list)  # Unit keys
    unit_sizes: List[int] = field(default_factory=list)  # Models per unit
    total_points: int = 0
    fitness: float = 0.0
    
    def __post_init__(self):
        self.calculate_points()
    
    def calculate_points(self):
        """Calculate total army points"""
        self.total_points = 0
        nuln_unit_templates = create_nuln_units()
        
        for unit_key, size in zip(self.units, self.unit_sizes):
            if unit_key in nuln_unit_templates:
                unit_template = nuln_unit_templates[unit_key]
                if unit_template.unit_type == UnitType.CHARACTER:
                    self.total_points += unit_template.points_cost
                else:
                    self.total_points += unit_template.points_cost * size
    
    def to_army_list(self) -> List[Unit]:
        """Convert composition to actual Unit objects"""
        army = []
        nuln_unit_templates = create_nuln_units()
        
        for unit_key, size in zip(self.units, self.unit_sizes):
            if unit_key in nuln_unit_templates:
                unit_template = copy.deepcopy(nuln_unit_templates[unit_key])
                if unit_template.unit_type != UnitType.CHARACTER:
                    unit_template.models_count = size
                    unit_template.current_models = size
                army.append(unit_template)
        
        return army
    
    def is_valid(self, points_limit: int = 1000) -> bool:
        """Check if army composition is valid"""
        # Check points limit
        if self.total_points > points_limit:
            return False
        
        # Check force organization requirements (simplified)
        has_core = any(unit_key.startswith("nuln_state_troops") for unit_key in self.units)
        if not has_core:
            return False
        
        return True

class GeneticAlgorithm:
    """Genetic Algorithm for evolving Nuln army compositions"""
    
    def __init__(self, population_size: int = 30, generations: int = 20, 
                 mutation_rate: float = 0.1, crossover_rate: float = 0.8,
                 points_limit: int = 1000):
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.points_limit = points_limit
        
        # Available Nuln units
        self.available_units = list(create_nuln_units().keys())
        self.unit_templates = create_nuln_units()
        
        # Evolution tracking
        self.best_fitness_history = []
        self.avg_fitness_history = []
    
    def create_random_army(self) -> ArmyComposition:
        """Create a random valid army composition"""
        max_attempts = 100
        
        for _ in range(max_attempts):
            units = []
            unit_sizes = []
            
            # Always include at least one core unit
            core_units = [unit for unit in self.available_units 
                         if unit.startswith("nuln_state_troops")]
            if core_units:
                chosen_core = random.choice(core_units)
                units.append(chosen_core)
                unit_sizes.append(random.randint(5, 15))
            
            # Add random other units
            num_additional_units = random.randint(1, 4)
            for _ in range(num_additional_units):
                unit_key = random.choice(self.available_units)
                
                if unit_key not in units:  # Avoid duplicates
                    units.append(unit_key)
                    
                    # Determine unit size
                    unit_template = self.unit_templates[unit_key]
                    if unit_template.unit_type == UnitType.CHARACTER:
                        unit_sizes.append(1)
                    else:
                        unit_sizes.append(random.randint(1, 20))
            
            composition = ArmyComposition(units, unit_sizes)
            if composition.is_valid(self.points_limit):
                return composition
        
        # Fallback: minimal valid army
        return ArmyComposition(
            units=["nuln_state_troops_handgun"],
            unit_sizes=[10]
        )
    
    def initialize_population(self) -> List[ArmyComposition]:
        """Create initial population"""
        population = []
        for _ in range(self.population_size):
            army = self.create_random_army()
            population.append(army)
        return population
    
    def evaluate_fitness(self, army: ArmyComposition, games_per_evaluation: int = 5) -> float:
        """Evaluate army fitness by playing games"""
        if army.fitness > 0:  # Already evaluated
            return army.fitness
        
        army_list = army.to_army_list()
        enemy_army = self.create_standard_enemy()
        
        wins = 0
        total_games = 0
        
        for _ in range(games_per_evaluation):
            # Reset army states
            for unit in army_list:
                unit.current_wounds = unit.wounds
                unit.current_models = unit.models_count
                unit.position = (random.randint(5, 15), random.randint(5, 15))
            
            for unit in enemy_army:
                unit.current_wounds = unit.wounds
                unit.current_models = unit.models_count
                unit.position = (random.randint(30, 40), random.randint(30, 40))
            
            # Create game state
            board = Board()
            game_state = GameState(
                board=board,
                player1_army=copy.deepcopy(army_list),
                player2_army=copy.deepcopy(enemy_army)
            )
            
            # Play game with MCTS vs Heuristic
            mcts_agent = MCTSAgent(iterations=100)  # Reduced for speed
            heuristic_agent = HeuristicAgent()
            
            winner = self.play_game(game_state, mcts_agent, heuristic_agent)
            
            if winner == 1:  # MCTS (our army) wins
                wins += 1
            total_games += 1
        
        army.fitness = wins / total_games if total_games > 0 else 0.0
        return army.fitness
    
    def create_standard_enemy(self) -> List[Unit]:
        """Create a standard enemy army for testing"""
        enemy_templates = create_enemy_units()
        enemy_army = [
            copy.deepcopy(enemy_templates["orc_warriors"]),
            copy.deepcopy(enemy_templates["orc_warboss"])
        ]
        return enemy_army
    
    def play_game(self, game_state: GameState, agent1, agent2, max_turns: int = 50) -> Optional[int]:
        """Play a game between two agents"""
        turn_count = 0
        
        while not game_state.is_game_over() and turn_count < max_turns:
            current_agent = agent1 if game_state.current_player == 1 else agent2
            
            try:
                action = current_agent.get_best_action(game_state)
                game_state = action.execute(game_state)
            except Exception as e:
                # If action fails, end phase
                game_state.advance_phase()
            
            turn_count += 1
        
        return game_state.winner
    
    def tournament_selection(self, population: List[ArmyComposition], tournament_size: int = 3) -> ArmyComposition:
        """Tournament selection for parent selection"""
        tournament = random.sample(population, min(tournament_size, len(population)))
        return max(tournament, key=lambda x: x.fitness)
    
    def crossover(self, parent1: ArmyComposition, parent2: ArmyComposition) -> Tuple[ArmyComposition, ArmyComposition]:
        """Crossover between two army compositions"""
        if random.random() > self.crossover_rate:
            return copy.deepcopy(parent1), copy.deepcopy(parent2)
        
        # Simple crossover: swap some units
        child1_units = parent1.units.copy()
        child1_sizes = parent1.unit_sizes.copy()
        child2_units = parent2.units.copy()
        child2_sizes = parent2.unit_sizes.copy()
        
        # Swap random portion
        if len(child1_units) > 1 and len(child2_units) > 1:
            split_point = random.randint(1, min(len(child1_units), len(child2_units)) - 1)
            
            # Swap tails
            temp_units = child1_units[split_point:]
            temp_sizes = child1_sizes[split_point:]
            
            child1_units = child1_units[:split_point] + child2_units[split_point:]
            child1_sizes = child1_sizes[:split_point] + child2_sizes[split_point:]
            
            child2_units = child2_units[:split_point] + temp_units
            child2_sizes = child2_sizes[:split_point] + temp_sizes\n        \n        child1 = ArmyComposition(child1_units, child1_sizes)\n        child2 = ArmyComposition(child2_units, child2_sizes)\n        \n        return child1, child2\n    \n    def mutate(self, army: ArmyComposition) -> ArmyComposition:\n        \"\"\"Mutate an army composition\"\"\"\n        if random.random() > self.mutation_rate:\n            return army\n        \n        mutated = copy.deepcopy(army)\n        \n        # Types of mutations\n        mutation_type = random.choice([\"change_unit\", \"change_size\", \"add_unit\", \"remove_unit\"])\n        \n        if mutation_type == \"change_unit\" and mutated.units:\n            # Replace a random unit\n            idx = random.randint(0, len(mutated.units) - 1)\n            # Don't change core units to maintain validity\n            if not mutated.units[idx].startswith(\"nuln_state_troops\"):\n                mutated.units[idx] = random.choice(self.available_units)\n        \n        elif mutation_type == \"change_size\" and mutated.unit_sizes:\n            # Change size of a random unit\n            idx = random.randint(0, len(mutated.unit_sizes) - 1)\n            unit_key = mutated.units[idx]\n            unit_template = self.unit_templates[unit_key]\n            \n            if unit_template.unit_type != UnitType.CHARACTER:\n                mutated.unit_sizes[idx] = random.randint(1, 20)\n        \n        elif mutation_type == \"add_unit\" and len(mutated.units) < 6:\n            # Add a new unit\n            new_unit = random.choice(self.available_units)\n            if new_unit not in mutated.units:\n                mutated.units.append(new_unit)\n                unit_template = self.unit_templates[new_unit]\n                if unit_template.unit_type == UnitType.CHARACTER:\n                    mutated.unit_sizes.append(1)\n                else:\n                    mutated.unit_sizes.append(random.randint(1, 15))\n        \n        elif mutation_type == \"remove_unit\" and len(mutated.units) > 1:\n            # Remove a unit (but not core units)\n            non_core_indices = [i for i, unit in enumerate(mutated.units) \n                              if not unit.startswith(\"nuln_state_troops\")]\n            if non_core_indices:\n                idx = random.choice(non_core_indices)\n                del mutated.units[idx]\n                del mutated.unit_sizes[idx]\n        \n        mutated.calculate_points()\n        mutated.fitness = 0.0  # Reset fitness\n        \n        return mutated\n    \n    def evolve(self, verbose: bool = True) -> ArmyComposition:\n        \"\"\"Run the genetic algorithm\"\"\"\n        # Initialize population\n        population = self.initialize_population()\n        \n        if verbose:\n            print(f\"Starting evolution with population size {self.population_size}\")\n            print(f\"Running for {self.generations} generations\")\n        \n        for generation in range(self.generations):\n            # Evaluate fitness\n            for army in population:\n                self.evaluate_fitness(army)\n            \n            # Sort by fitness\n            population.sort(key=lambda x: x.fitness, reverse=True)\n            \n            # Track statistics\n            best_fitness = population[0].fitness\n            avg_fitness = sum(army.fitness for army in population) / len(population)\n            \n            self.best_fitness_history.append(best_fitness)\n            self.avg_fitness_history.append(avg_fitness)\n            \n            if verbose:\n                print(f\"Generation {generation + 1}: Best fitness = {best_fitness:.3f}, Avg = {avg_fitness:.3f}\")\n                print(f\"  Best army: {population[0].units} (sizes: {population[0].unit_sizes})\")\n                print(f\"  Points: {population[0].total_points}\")\n            \n            # Create next generation\n            new_population = []\n            \n            # Elitism: keep best individuals\n            elite_count = max(1, self.population_size // 10)\n            new_population.extend(copy.deepcopy(population[:elite_count]))\n            \n            # Generate offspring\n            while len(new_population) < self.population_size:\n                parent1 = self.tournament_selection(population)\n                parent2 = self.tournament_selection(population)\n                \n                child1, child2 = self.crossover(parent1, parent2)\n                \n                child1 = self.mutate(child1)\n                child2 = self.mutate(child2)\n                \n                # Only add valid children\n                if child1.is_valid(self.points_limit):\n                    new_population.append(child1)\n                if child2.is_valid(self.points_limit) and len(new_population) < self.population_size:\n                    new_population.append(child2)\n                \n                # Fill with random if needed\n                if len(new_population) < self.population_size:\n                    new_population.append(self.create_random_army())\n            \n            population = new_population[:self.population_size]\n        \n        # Final evaluation\n        for army in population:\n            self.evaluate_fitness(army)\n        \n        population.sort(key=lambda x: x.fitness, reverse=True)\n        return population[0]\n\nprint(\"Genetic Algorithm implemented!\")"


In [ ]:
# Test the game simulator with a simple battle
def test_game_simulator():
    print("=== Testing Game Simulator ===")
    
    # Create armies
    nuln_templates = create_nuln_units()
    enemy_templates = create_enemy_units()
    
    nuln_army = [
        copy.deepcopy(nuln_templates["nuln_state_troops_handgun"]),
        copy.deepcopy(nuln_templates["nuln_great_cannon"])
    ]
    
    enemy_army = [
        copy.deepcopy(enemy_templates["orc_warriors"])
    ]
    
    # Set positions
    nuln_army[0].position = (10, 10)
    nuln_army[1].position = (15, 10)
    enemy_army[0].position = (30, 30)
    
    # Create game state
    board = Board()
    game_state = GameState(
        board=board,
        player1_army=nuln_army,
        player2_army=enemy_army
    )
    
    print(f"Initial state:")
    print(f"  Nuln Army: {len(game_state.get_alive_units(nuln_army))} units")
    print(f"  Enemy Army: {len(game_state.get_alive_units(enemy_army))} units")
    
    # Test a shooting attack
    shooter = nuln_army[0]
    target = enemy_army[0]
    distance = board.distance(shooter.position, target.position)
    
    print(f"\\nTesting shooting attack:")
    print(f"  Distance: {distance:.1f} inches")
    print(f"  Target models before: {target.current_models}")
    
    wounds = GameMechanics.shooting_attack(shooter, target, distance)
    target.take_wounds(wounds)
    
    print(f"  Wounds caused: {wounds}")
    print(f"  Target models after: {target.current_models}")
    
    return game_state

# Test basic game
test_game_state = test_game_simulator()


In [ ]:
# Test MCTS Agent vs Heuristic Agent
def test_agents():
    print("\\n=== Testing AI Agents ===")
    
    # Create a simple game
    nuln_templates = create_nuln_units()
    enemy_templates = create_enemy_units()
    
    nuln_army = [copy.deepcopy(nuln_templates["nuln_state_troops_handgun"])]
    enemy_army = [copy.deepcopy(enemy_templates["orc_warriors"])]
    
    # Set positions
    nuln_army[0].position = (10, 10)
    enemy_army[0].position = (35, 35)
    
    board = Board()
    game_state = GameState(
        board=board,
        player1_army=nuln_army,
        player2_army=enemy_army
    )
    
    # Create agents
    mcts_agent = MCTSAgent(iterations=50)  # Reduced for demo
    heuristic_agent = HeuristicAgent()
    
    print(f"Game setup:")
    print(f"  MCTS Agent (Player 1): {len(nuln_army)} units")
    print(f"  Heuristic Agent (Player 2): {len(enemy_army)} units")
    
    # Play a few turns
    turns_played = 0
    max_demo_turns = 10
    
    while not game_state.is_game_over() and turns_played < max_demo_turns:
        current_agent = mcts_agent if game_state.current_player == 1 else heuristic_agent
        agent_name = "MCTS" if game_state.current_player == 1 else "Heuristic"
        
        print(f"\\nTurn {turns_played + 1} - {agent_name} Agent (Player {game_state.current_player}):")
        print(f"  Phase: {game_state.current_phase.value}")
        
        try:
            action = current_agent.get_best_action(game_state)
            print(f"  Action: {action.description()}")
            game_state = action.execute(game_state)
        except Exception as e:
            print(f"  Error: {e}")
            game_state.advance_phase()
        
        turns_played += 1
    
    print(f"\\nDemo completed after {turns_played} turns")
    if game_state.is_game_over():
        print(f"Winner: Player {game_state.winner}")
    else:
        print("Game still in progress")

# Test the agents
test_agents()


In [ ]:
# Demo the Genetic Algorithm (Quick Version)
def demo_genetic_algorithm():
    print("\\n=== Genetic Algorithm Demo ===")
    print("Running a quick evolution with reduced parameters for demonstration...")
    
    # Create GA with reduced parameters for demo
    ga = GeneticAlgorithm(
        population_size=10,  # Small for demo
        generations=5,       # Quick demo
        mutation_rate=0.15,
        crossover_rate=0.8,
        points_limit=500     # Smaller armies for speed
    )
    
    print(f"\\nGA Configuration:")
    print(f"  Population Size: {ga.population_size}")
    print(f"  Generations: {ga.generations}")
    print(f"  Points Limit: {ga.points_limit}")
    print(f"  Available Units: {ga.available_units}")
    
    # Test army creation
    print(f"\\nTesting random army generation:")
    test_army = ga.create_random_army()
    print(f"  Units: {test_army.units}")
    print(f"  Sizes: {test_army.unit_sizes}")
    print(f"  Total Points: {test_army.total_points}")
    print(f"  Is Valid: {test_army.is_valid(ga.points_limit)}")
    
    # Run evolution
    print(f"\\nStarting evolution...")
    best_army = ga.evolve(verbose=True)
    
    print(f"\\n=== EVOLUTION COMPLETE ===")
    print(f"Best Army Found:")
    print(f"  Units: {best_army.units}")
    print(f"  Sizes: {best_army.unit_sizes}")
    print(f"  Total Points: {best_army.total_points}")
    print(f"  Fitness (Win Rate): {best_army.fitness:.3f}")
    
    return ga, best_army

# Run the demo (this will take a few minutes)
print("Starting Genetic Algorithm demo...")
print("Note: This will take a few minutes as it runs multiple game simulations...")

ga_demo, best_army_result = demo_genetic_algorithm()


In [ ]:
# Plotting evolution results
def plot_evolution_results(ga):
    """Plot the evolution progress"""
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(ga.best_fitness_history, 'b-', linewidth=2, label='Best Fitness')
    plt.plot(ga.avg_fitness_history, 'r--', linewidth=2, label='Average Fitness')
    plt.xlabel('Generation')
    plt.ylabel('Fitness (Win Rate)')
    plt.title('Evolution Progress')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.bar(range(len(ga.best_fitness_history)), ga.best_fitness_history, alpha=0.7, color='blue')
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness')
    plt.title('Best Fitness by Generation')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot results if we have evolution data
if 'ga_demo' in locals() and ga_demo.best_fitness_history:
    plot_evolution_results(ga_demo)
else:
    print("Evolution results not available for plotting")

# Summary and analysis
def analyze_results(best_army):
    """Analyze the best army composition found"""
    print("\\n=== ANALYSIS OF BEST ARMY ===")
    
    nuln_templates = create_nuln_units()
    
    print("\\nDetailed Unit Analysis:")
    total_models = 0
    ranged_units = 0
    melee_units = 0
    artillery_units = 0
    
    for unit_key, size in zip(best_army.units, best_army.unit_sizes):
        if unit_key in nuln_templates:
            unit = nuln_templates[unit_key]
            print(f"\\n  {unit.name}:")
            print(f"    Models: {size}")
            print(f"    Points: {unit.points_cost * size if unit.unit_type != UnitType.CHARACTER else unit.points_cost}")
            print(f"    Type: {unit.unit_type.value}")
            
            total_models += size
            
            # Categorize by role
            has_ranged = any(eq.weapon_type == WeaponType.RANGED for eq in unit.equipment)
            has_artillery = any(eq.weapon_type == WeaponType.ARTILLERY for eq in unit.equipment)
            
            if has_artillery:
                artillery_units += 1
            elif has_ranged:
                ranged_units += 1
            else:
                melee_units += 1
    
    print(f"\\nArmy Composition Summary:")
    print(f"  Total Models: {total_models}")
    print(f"  Ranged Units: {ranged_units}")
    print(f"  Melee Units: {melee_units}")
    print(f"  Artillery Units: {artillery_units}")
    print(f"  Total Points: {best_army.total_points}")
    print(f"  Win Rate: {best_army.fitness:.1%}")
    
    # Strategic analysis
    print(f"\\nStrategic Analysis:")
    if ranged_units > melee_units:
        print("  ✓ Ranged-focused army: Good for Nuln's shooting strengths")
    if artillery_units > 0:
        print("  ✓ Artillery support: Leverages Nuln's technological advantages")
    if total_models < 20:
        print("  ✓ Elite army: Quality over quantity approach")
    else:
        print("  ✓ Numerous army: Strength in numbers")

# Analyze the results
if 'best_army_result' in locals():
    analyze_results(best_army_result)
else:
    print("Best army results not available for analysis")


In [ ]:
# Final summary and usage instructions
print("🎯 WARHAMMER: THE OLD WORLD AI SYSTEM READY!")
print("=" * 50)
print()
print("📋 QUICK START GUIDE:")
print()
print("1. BASIC SIMULATION:")
print("   - Run test_game_simulator() to see combat mechanics")
print("   - Modify unit positions and test different scenarios")
print()
print("2. AI AGENT TESTING:")
print("   - Use test_agents() to watch MCTS vs Heuristic battles")
print("   - Adjust MCTS iterations for different AI strength")
print()
print("3. ARMY OPTIMIZATION:")
print("   - Run demo_genetic_algorithm() for quick evolution")
print("   - For serious optimization, increase population_size and generations")
print()
print("4. CUSTOM EXPERIMENTS:")
print("   - Add new Nuln units to create_nuln_units()")
print("   - Modify enemy_units for different opponents")
print("   - Adjust GA parameters for different evolution strategies")
print()
print("⚙️  PERFORMANCE TIPS:")
print("   - Reduce MCTS iterations for faster gameplay")
print("   - Use smaller armies (lower points limits) for quicker evolution")
print("   - Increase games_per_evaluation for more accurate fitness")
print()
print("🔬 RESEARCH OPPORTUNITIES:")
print("   - Compare different opponent types")
print("   - Analyze meta-game trends")
print("   - Test scenario-specific army builds")
print("   - Develop counter-strategies")
print()
print("Ready to optimize your Nuln armies! 🛡️⚔️")

# Save the current state for later use
import pickle

def save_results(ga, best_army, filename="warhammer_ai_results.pkl"):
    """Save GA results for later analysis"""
    results = {
        'best_army': best_army,
        'best_fitness_history': ga.best_fitness_history,
        'avg_fitness_history': ga.avg_fitness_history,
        'ga_parameters': {
            'population_size': ga.population_size,
            'generations': ga.generations,
            'mutation_rate': ga.mutation_rate,
            'crossover_rate': ga.crossover_rate,
            'points_limit': ga.points_limit
        }
    }
    
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    
    print(f"Results saved to {filename}")

# Save results if available
if 'ga_demo' in locals() and 'best_army_result' in locals():
    save_results(ga_demo, best_army_result)
    
print("\n✅ Notebook setup complete! All systems operational.")
